# CNN Cancer Detection Kaggle Mini-Project

##  Brief Description of The Problem and Data 

This project is part of a Kaggle competition aimed at automatically detecting metastatic cancer in small image patches extracted from larger digital pathology scans. The goal is to develop a binary classifier that can predict whether a given image patch contains cancerous tissue.